<center> <h1> Restoration of underwater images </h1> <br> <h3> <br> 4MEO Computer vision 2</h3> </center>

# I. Introduction

### 1. Import libraries

In [5]:
import numpy as np
import scipy as sc
import cv2
import matplotlib.image as mpimg
import matplotlib.pylab as plt

# II. Compute inputs

In [6]:
firstInput = cv2.imread("data/test1.png", 0)
wb = cv2.xphoto.createGrayworldWB()
wb.setSaturationThreshold(0.99)
firstInput = wb.balanceWhite(firstInput)

error: OpenCV(4.5.4) /Users/runner/work/opencv-python/opencv-python/opencv_contrib/modules/xphoto/src/grayworld_white_balance.cpp:63: error: (-215:Assertion failed) !_src.empty() in function 'balanceWhite'
